# word2vec 단어 간 유사도 체크
- 최대한 육아를 주제로 한 말뭉치를 모아보았다. (Total 2,755 documents)
- 우리가 원하는 의도에 일치하는 단어 간 유사도 결과가 나오는지를 확인하는 것이 목적이다.

## Import

In [1]:
import pandas as pd
import numpy as np
import re

from konlpy.tag import Mecab
from konlpy.tag import Komoran
from gensim.models import Word2Vec

### Datas

In [2]:
df1 = pd.read_csv("data/maeili.csv", encoding='utf-8')
df2 = pd.read_csv("data/baby.csv", encoding='utf-8')
df3 = pd.read_csv("data/toys.csv", encoding='utf-8')
df4 = pd.read_csv("data/seoulchildcare.csv", encoding='utf-8')
df5 = pd.read_csv("data/babynews_DLC.csv", encoding='utf-8')
df6 = pd.read_csv("data/ildongfoodis.csv", encoding='utf-8')
df7 = pd.read_csv("data/momsmagazine.csv", encoding='utf-8')
df8 = pd.read_csv("data/ange.csv", encoding='utf-8')
df9 = pd.read_csv("data/babytimes.csv", encoding='utf-8')
df10 = pd.read_csv("data/daekyo.csv", encoding='utf-8')

#### maeili(매일아이)

In [3]:
df1.tail()

,categories,title,text,postlink
1533,임신,'웃음요법' 태교,임신 중에는 스트레스를 받지 않고 몸을 편안하게 하는 것만큼 좋은 태교가 없습니다 ...,https://www.maeili.com/cms/contents/contentsVi...
1534,임신,기형 검사와 보조 검사,모든 임신부는 나이에 상관없이 7백 명에 한 명꼴로 정신박약을 초래하는 다운 증후군...,https://www.maeili.com/cms/contents/contentsVi...
1535,임신,임신 시기별 검사 종류와 횟수,첫방문시 혈액검사 소변검사 혈액형 검사 풍진항체 검사 8 18주 초음파검사 필요한...,https://www.maeili.com/cms/contents/contentsVi...
1536,임신,초음파 검사의 임상적 유용성,초기의 정상적인 자궁내 임신의 확인 임신낭과 태아의 성장 여부와 크기 측정 임신주...,https://www.maeili.com/cms/contents/contentsVi...
1537,임신,산전 유전 상담이란,유전 질환 염색체 이상도 말초 혈액의 염색체 검사로서 진단이 가능하나 산과 영역에서...,https://www.maeili.com/cms/contents/contentsVi...


#### momtalk(육아정보)

In [4]:
df2.tail()

,idx,text
695,852,유아용품 유통기한 바로알기 기저귀 화장품 기저귀 화장품에도 유통기한이 있다구 두고...
696,928,우리 아이 쑥쑥 클 수 있게 전지현 크림으로 하는 베이비 마사지법 서 알려드려요 ...
697,971,이유식을 시작하는 맘들을 위해 이유식 준비물 요점 정리 이유식을 시작하는 맘들을 ...
698,843,휴일에 문 여는 약국이 궁금하다면 휴일지킴이약국 검색법 하필 쉬는 날 응급약이 필...
699,842,우리 아이가 아파요 응급실에 가야할까요 소아과 전문의 김연호 선생님과 함께하는 우...


#### momtalk(유아용품)

In [5]:
df3.tail()

,idx,text
482,622,우리아이 치발기 치아발육기 선택 꿀팁 잇몸이 근질근질우리아이 치아발육기 선택팁 아...
483,640,쏘서 점퍼루 보행기 우리아이 선택은 쏘서 점퍼루 보행기 우리아이 선택은 아이를 낳...
484,607,신생아부터 유아까지 아기목욕용품 및 아기 목욕시키기 신생아부터 유아까지아기 목욕용...
485,567,아기용품 어떤걸로 골라야 할까 아기용품 고르기 오늘은 아기용품고르기에 대해 함께 ...
486,604,장난감수납장 이거좋아요 이번주 베스트글 주인공은 소영님 입니다 요즘 장난감수납장에...


#### 서울육아종합지원센터 육아전문칼럼

In [6]:
df4.tail()

,text
35,좋은 아빠 지금 하지 않으면 늦다 결혼을 하고 아이를 키우면서 가장 힘든 점은 욕구...
36,놀이의 뇌과학적 가치 요즘 아이들은 밖에서 같이 놀 기회가 적어졌다 누군가와 놀아도...
37,영유아기 자녀와 안정 애착 형성하기 영유아기 때 부모와 맺은 관계 경험은 평생을 통...
38,아이를 숲으로 데려갈까요 4차 산업혁명시대를 맞이하여 더 많이 놀게 하고 더 많이 ...
39,NaN


#### 육아신문 베이비뉴스

In [7]:
df5.tail()

,titles,text,postlink
9679,"함께 놀고 싶고, 함께 일하고 싶은 아이로 키우기",Q 영·유아기에 친구들이 함께 놀고 싶어하는 아이라면 성인이 됐을 때 함께 일하고 ...,https://www.ibabynews.com/news/articleView.htm...
9680,"아빠 육아, 왜 하필 사회성 발달에 좋다는 걸까?",Q 최근 대중매체에서 아빠육아를 자주 언급하고 실제 방송프로그램을 보면 엄마 없이 ...,https://www.ibabynews.com/news/articleView.htm...
9681,"미국에서 육아휴직하기, 돌봄서비스 찾기",첫째를 가지고 아이를 전적으로 내 힘으로만 키워보겠다는 다짐과 의욕에 넘치던 기간이...,https://www.ibabynews.com/news/articleView.htm...
9682,"""잠 못 자는 아이들, 어른들이 미웠다""",아 정말 강남귀신이라도 만났으면 좋겠다 그럼 좀 시원하지 않을까정말이지 너무 덥다 ...,https://www.ibabynews.com/news/articleView.htm...
9683,"""사람은 안 변한다?"" 인간을 변화 시키는 방법",어후 저 인간 저거 언제 사람 되려나 제가 어린 시절 집에서 많이 듣던 말이었습니다...,https://www.ibabynews.com/news/articleView.htm...


#### 일동후디스

In [8]:
df6.tail()

,titles,text,postlinks
1640,육아에 필요한 법률상담,고귀한 탯줄법률상담5살인 저희 아이는 심각한 고등어 알러지가 있습니다 그래서 집에...,http://foodis.com/web/www/baby_column_02;jsess...
1641,이럴 땐 이렇게 하세요,아기를 가진 엄마들의 궁금증현재 두 아기의 엄마이기도 한종로구 의 김현아 선생님께서...,http://foodis.com/web/www/baby_column_02;jsess...
1642,"키 작은 아이, 남들만큼 클 수 없을까요?",키 작은 아이 남들만큼 클 수 없을까요 먹거리가 귀한 시절에 부모는 아이가 단지...,http://foodis.com/web/www/baby_column_02;jsess...
1643,스킨쉽을 통해 엄마의 사랑을 전해주세요.,마사지를 시작하기 전에 1 항상 시작신호를 보내고 아기의 의사를 존중해야한다 ...,http://foodis.com/web/www/baby_column_02;jsess...
1644,전문의에게 배우는 초보 엄마 아기 도우미,초보 엄마 도우미전문의에게 배우는 초보 엄마 아기 도우미우리 아기 응급 처치법 아...,http://foodis.com/web/www/baby_column_02;jsess...


#### 맘스매거진

In [9]:
df7.tail()

,titles,text,postlink
387,‘믿지 마세요.’ 육아 속설들,육아에 대한 말과 정보들은 많지만 근거 없는 육아 속설에 의지해 아이를 돌보면 큰 ...,http://momsmagazine.co.kr/%eb%af%bf%ec%a7%80-%...
388,신생아가 있는 집 에어컨 사용법,신생아는 땀구멍이 열리지 않아 땀을 흘리지 않는다는 이유로 더위를 타지 않을 거라고...,http://momsmagazine.co.kr/%ec%8b%a0%ec%83%9d%e...
389,초등학교 입학 전 읽기 능력을 키우는 법,초등학교 입학을 앞둔 있는 엄마들은 내 아이가 한글을 읽고 쓰는 데 어려움이 없지 ...,http://momsmagazine.co.kr/%ec%b4%88%eb%93%b1%e...
390,아는 만큼 혜택 보는 2015 출산 지원 제도,2015년 정부는 임신과 출산을 위한 지원제도를 마련했다 임산부에게 경제적 심리적 ...,http://momsmagazine.co.kr/%ec%95%84%eb%8a%94-%...
391,우리 아기 여름 땀띠 똑똑하게 관리하기,덥고 습한 여름은 민감한 아기 피부에 트러블이 생기기 쉽다 영유아는 어른보다 땀샘의...,http://momsmagazine.co.kr/%ec%9a%b0%eb%a6%ac-%...


#### 앙쥬

In [10]:
df8.tail()

,titles,text,postlink
2853,"장거리 차량 이동, 보채는 아이 상황별 대처법",아이는 현재 신체와 정신이 모두 발달 중에 있다 신체 발달은 근육의 발달을 정신 발...,https://www.ange.co.kr/usr/?menu=story602&subm...
2854,"어느 날, 아이가 밥상머리 전쟁을 선포했다",세 살 적 버릇이 여든까지 간다는 말이 있다 밥상머리 예절도 예외는 아니다 아이에게...,https://www.ange.co.kr/usr/?menu=story602&subm...
2855,"발달 늦는 아이, 기다릴 것인가? 말 것인가?",아이의 키가 자라고 몸무게가 늘어나면서 아이 몸을 구성하고 있는 장기들도 커지고 무...,https://www.ange.co.kr/usr/?menu=story602&subm...
2856,전전긍긍! 내 아이가 내 눈치를 본다? 도대체 왜?,눈치는 직관적이고 비언어적인 방법으로 의사소통을 하고 문제를 해결하는 놀라운 능력이...,https://www.ange.co.kr/usr/?menu=story602&subm...
2857,"아이의 소리,말 자극에 대한 보고서",소리를 들으면 사람은 그 소리를 그대로 흉내 낼 수 있다 이것이 청력의 힘이다 말소...,https://www.ange.co.kr/usr/?menu=story602&subm...


#### 베이비타임즈

In [11]:
df9.tail()

,titles,text,postlink
2441,"아이돌봄서비스, 온라인으로 신청하세요",보건복지부는 보육료 및 교육비 신청을 받아오던 온라인신청 새롭게 확대 개편해 여가부...,http://www.babytimes.co.kr/news/articleView.ht...
2442,영유아 감염병 ‘성홍열’ 이상 급증,혓바닥이 빨갛고 오톨도톨해지며 딸기모양으로 보이는 영유아 감염병 성홍열이 원인이 파...,http://www.babytimes.co.kr/news/articleView.ht...
2443,"키즈카페, '청결하지 못한 위생 상태' 불만",키즈카페를 이용하는 부모들은 카페의 청결하지 못한 위생 상태를 가장 큰 불만으로 꼽...,http://www.babytimes.co.kr/news/articleView.ht...
2444,"어린이 정기예방접종, 2014년부터 전면 무료",보건복지부와 질병관리본부는 내년 1월부터 만 12세 이하 어린이의 정기예방접종이 전...,http://www.babytimes.co.kr/news/articleView.ht...
2445,세종청사 어린이집 CCTV 보도… 누리꾼 분노,정부세종청사 내 어린이집 아동학대 사건의 전말을 보도한 언론 보도 이후 누리꾼들이 ...,http://www.babytimes.co.kr/news/articleView.ht...


#### 대교

In [12]:
df10.tail()

,titles,text,postlink
794,"[교육을 말한다] 우리 아이에게 맞는 학습법, 띠 별 아이 학습법③",토끼띠의 단점,https://blog.daekyo.com/582?category=584155
795,[교육을 말한다] 국가영어능력평가 제대로 알고 준비해봐요!!,국가영어능력평가 준비 올해부터 본격적으로 시행되는 국가영어능력평가에 대해서 소개해드...,https://blog.daekyo.com/568?category=584155
796,"[교육을 말한다] 워킹맘 스토리, 일을 해야 하나요? 말아야 하나요?",워킹맘이든 전업맘이든 아이들에게 관심과 사랑을 쏟아야 하는 것은 분명합니다 평소 아...,https://blog.daekyo.com/564?category=584155
797,[진로상담센터] 아이들의 고민을 함께 나눠요,검사결과 지민이의 강점지능은 언어지능과 음악지능였습니다 엄마는 지민이가 집에서나 학...,https://blog.daekyo.com/539?category=584155
798,[진로상담센터] 초등학생 진로 상담 교육 컨설팅,진로상담센터 교육 컨설팅 아이들의 진로는 중·고등학생 때 뿐 아니라 초등학생때부터 ...,https://blog.daekyo.com/507?category=584155


### Concatenate Dataframe

In [13]:
df = pd.concat([df1.text, df2.text, df3.text, df4.text, df5.text,
                df6.text, df7.text, df8.text, df9.text, df10.text])
df = pd.DataFrame(df).reset_index(drop=True)
df.tail()

,text
20584,토끼띠의 단점
20585,국가영어능력평가 준비 올해부터 본격적으로 시행되는 국가영어능력평가에 대해서 소개해드...
20586,워킹맘이든 전업맘이든 아이들에게 관심과 사랑을 쏟아야 하는 것은 분명합니다 평소 아...
20587,검사결과 지민이의 강점지능은 언어지능과 음악지능였습니다 엄마는 지민이가 집에서나 학...
20588,진로상담센터 교육 컨설팅 아이들의 진로는 중·고등학생 때 뿐 아니라 초등학생때부터 ...


### Preprocessing

In [14]:
def cleaningText(text):
    text = re.sub('&nbsp;|\xa0|\u200b|\r|\n|\t', ' ', text)
    text = re.sub('<.+>|【.+】|\[.+\]', ' ', text)
    text = re.sub('[-_=+,/\?;:^$.@*\"※~&%ㆍ!”“’「」『』〈〉\\‘|\(\)\{\}\[\]\<\>`\'…《》]', ' ', text)
    text = re.sub('[㈜×–～→▪‧•∘○□◇△▽▷◁☆◈◐▶▼▲◆★♦■♥️●♠🖤]', ' ', text)
    text = re.sub('[①②③④⑤⑥⑦⑧⑨⑩⑪⑫⑬⑭⑮⑯⑰⑱⑲⑳]', ' ', text)
    text = re.sub('[➊❷➌❹❺❻❼❽❾❿]', ' ', text)
    text = re.sub('http.+com|http.+co|http.+kr|www.+kr|www.+com|www.+net|http.+net', ' ', text)
    text = re.sub("\s.+\s기자", '', text)
    text = re.sub('ⓒ\S+\s|㈜\S+\s|홈페이지\S+\s|전화\S+\s|대표전화\S+\s|☎\S+\s|http\S+\s', '', text)
    
    text = re.sub('\s+', ' ', text)
    text = text.strip()

    return text

In [15]:
df = df.drop(index=[952, 955, 956, 960, 962, 964, 1165, 1240, 1322])
df = df.drop_duplicates()
df = df.dropna()
df = pd.DataFrame(df).reset_index(drop=True)

In [16]:
df['text'] = df['text'].apply(cleaningText)
df = df.dropna()
df.to_csv('data/text_for_word2vec.csv', index=False, encoding='utf-8')

In [17]:
df

,text
0,6살 남아 입니다 회사 복직 후 3살부터 주중에는 외할머니댁에서 2살 4살 터울의 ...
1,요즘 들어 아빠가 아이에게 다가가려고 하면 엄마한테 갈 거야 하면서 엄마를 유독 찾...
2,둘째가 태어나면 첫째가 힘들 거라는 건 예상하고 있었지만 툭하면 삐치고 울고 좀처럼...
3,20개월 남아가 있고 임신 8개월차로 두 아기의 엄마입니다 첫 아이는 잠자리 독립을...
4,14개월 남자 아기인데 심하게 깨무는 버릇이 있어요 전에는 졸릴 때 주로 물곤 했는...
...,...
19502,토끼띠의 단점
19503,국가영어능력평가 준비 올해부터 본격적으로 시행되는 국가영어능력평가에 대해서 소개해드...
19504,워킹맘이든 전업맘이든 아이들에게 관심과 사랑을 쏟아야 하는 것은 분명합니다 평소 아...
19505,검사결과 지민이의 강점지능은 언어지능과 음악지능였습니다 엄마는 지민이가 집에서나 학...


### Add tokenized column

191210 정체불명의 error 발생
```
NotImplementedError: Wrong number or type of arguments for overloaded function 'Tagger_parse'.
  Possible C/C++ prototypes are:
    MeCab::Tagger::parse(MeCab::Model const &,MeCab::Lattice *)
    MeCab::Tagger::parse(MeCab::Lattice *) const
    MeCab::Tagger::parse(char const *)
```

babynews 데이터가 추가된 후 갑자기 발생한 에러인데.. 왜인지 모르겠지만 해당 부분 데이터프레임만 따로 tokenize하는 것은 문제없이 돌아간다.  
따라서 번거롭지만 분리하여 tokenizing 한 후에 합치기로 하였다.  

-> 해결.. `NaN` value 문제였다(어이없게도 NaN 값이 중간에 섞여있었음;). 누락 데이터를 잘 체크하는 습관을 갖도록하자.

In [18]:
def tokenize(text):
    tokenizer = Mecab()
    text = tokenizer.morphs(text)
#     text = ' '.join(text)
    return text

In [19]:
%%time
df['tokenized'] = df['text'].apply(tokenize)
df.tail()

CPU times: user 41 s, sys: 22.7 s, total: 1min 3s
Wall time: 1min 3s


,text,tokenized
19502,토끼띠의 단점,"[토끼띠, 의, 단점]"
19503,국가영어능력평가 준비 올해부터 본격적으로 시행되는 국가영어능력평가에 대해서 소개해드...,"[국가, 영어, 능력, 평가, 준비, 올해, 부터, 본격, 적, 으로, 시행, 되,..."
19504,워킹맘이든 전업맘이든 아이들에게 관심과 사랑을 쏟아야 하는 것은 분명합니다 평소 아...,"[워킹, 맘, 이, 든, 전업, 맘, 이, 든, 아이, 들, 에게, 관심, 과, 사..."
19505,검사결과 지민이의 강점지능은 언어지능과 음악지능였습니다 엄마는 지민이가 집에서나 학...,"[검사, 결과, 지민, 이, 의, 강점, 지능, 은, 언어, 지능, 과, 음악, 지..."
19506,진로상담센터 교육 컨설팅 아이들의 진로는 중·고등학생 때 뿐 아니라 초등학생때부터 ...,"[진로, 상담, 센터, 교육, 컨설팅, 아이, 들, 의, 진로, 는, 중, ·, 고..."


In [20]:
df.isnull().sum()

text         0
tokenized    0
dtype: int64

## Train Word2Vec

#### WTF error
- df를 csv 형식으로 저장한 후 다시 로드하면 Word2Vec 학습에 인코딩 문제가 생긴다.
- 더 자세히 파고들어보니 list 형식의 column이 csv로 저장할 때 강제로 string 타입으로 형변환 됨.
- 이렇게 형변환 된 column을 Word2Vec 학습에 사용할 시 한글 텍스트가 깨지는 현상이 발생함.. 정확한 원인은 아직 모르겠다.

In [21]:
# df = pd.read_csv('text_for_word2vec.csv', encoding='utf-8')

그러므로 저장 & 로드 과정 생략하고 바로 말뭉치를 numpy ndarray형식으로 변환

In [22]:
corpus = np.array(df.tokenized)
type(corpus)

numpy.ndarray

In [23]:
model_all = Word2Vec(corpus, size=128, workers=4, sg=1)
model_all.save('data/tokmom_word2vec_all')

#### 유사어 분석

In [24]:
model_all.wv.similar_by_word("놀이")

[('소꿉', 0.7539886236190796),
 ('공놀이', 0.7020375728607178),
 ('놀잇감', 0.6859716176986694),
 ('놀이터', 0.6789550185203552),
 ('미술', 0.6679667234420776),
 ('역할극', 0.6658950448036194),
 ('음률', 0.6633263230323792),
 ('숨바꼭질', 0.6606078147888184),
 ('점토', 0.6601049900054932),
 ('찰흙', 0.6421756744384766)]

In [25]:
model_all.wv.similar_by_word("아이")

[('아기', 0.7134267687797546),
 ('어린아이', 0.7128130793571472),
 ('자녀', 0.6886004209518433),
 ('Solution', 0.6756339073181152),
 ('갓난아기', 0.6752938032150269),
 ('양손잡이', 0.671462893486023),
 ('본보기', 0.6544440984725952),
 ('외동아이', 0.6533958911895752),
 ('갓난아이', 0.651941180229187),
 ('큰아이', 0.6510705947875977)]

In [26]:
model_all.wv.similar_by_word("치아")

[('잇몸', 0.8231825232505798),
 ('영구치', 0.7919342517852783),
 ('충치', 0.7177867889404297),
 ('턱뼈', 0.7042288780212402),
 ('혀', 0.7022850513458252),
 ('유치', 0.6913747787475586),
 ('뼈', 0.6877995133399963),
 ('법랑질', 0.682285487651825),
 ('치근', 0.6756521463394165),
 ('치조골', 0.6698732376098633)]

In [27]:
model_all.wv.similar_by_word("이빨")

[('혓바닥', 0.8340562582015991),
 ('문다', 0.8209183216094971),
 ('핥', 0.8164038062095642),
 ('문질러서', 0.8072730302810669),
 ('근질거리', 0.8009214997291565),
 ('바싹', 0.8007936477661133),
 ('탁탁', 0.7960212230682373),
 ('뽀얗', 0.7951754331588745),
 ('군데군데', 0.7931393384933472),
 ('비눗물', 0.7916767597198486)]

In [28]:
model_all.wv.similar_by_word("감기")

[('코감기', 0.8362369537353516),
 ('냉방병', 0.78067946434021),
 ('독감', 0.7720118165016174),
 ('비염', 0.770840585231781),
 ('기관지염', 0.7630501389503479),
 ('목감기', 0.762562096118927),
 ('장염', 0.7495507001876831),
 ('호흡기', 0.7484935522079468),
 ('축농증', 0.7437417507171631),
 ('족구', 0.7378442287445068)]

In [29]:
model_all.wv.similar_by_word("열")

[('고열', 0.5873336791992188),
 ('3940', 0.5853402614593506),
 ('미열', 0.5769288539886475),
 ('고열로', 0.5744488835334778),
 ('오한', 0.5544812679290771),
 ('사병', 0.5527662634849548),
 ('일사병', 0.5512457489967346),
 ('외감', 0.5504831671714783),
 ('심열', 0.5399110317230225),
 ('성경', 0.5387053489685059)]

In [30]:
model_all.wv.similar_by_word("아파")

[('아파서', 0.8082139492034912),
 ('아프', 0.8022851943969727),
 ('아파하', 0.7990169525146484),
 ('다쳐', 0.7593474388122559),
 ('어지럽', 0.7576866745948792),
 ('보챈다', 0.756355881690979),
 ('가려워하', 0.7552841901779175),
 ('고픈지', 0.7506644129753113),
 ('아픈데', 0.7393842339515686),
 ('결리', 0.737580418586731)]

In [31]:
model_all.wv.similar_by_word("아프")

[('아파', 0.8022851943969727),
 ('어지럽', 0.7942240238189697),
 ('아파하', 0.773670494556427),
 ('결리', 0.7633020281791687),
 ('시리', 0.7509156465530396),
 ('땅기', 0.7439126372337341),
 ('간지럽', 0.7378921508789062),
 ('먹먹해', 0.7307586073875427),
 ('아팠', 0.7277988195419312),
 ('아파서', 0.7269823551177979)]

In [32]:
model_all.wv.similar_by_word("울음")

[('우', 0.6639549732208252),
 ('옹알이', 0.6573118567466736),
 ('소리', 0.6540652513504028),
 ('터뜨립니다', 0.6522349119186401),
 ('터뜨린다', 0.6496139764785767),
 ('터뜨려', 0.6489412784576416),
 ('운다', 0.6467686891555786),
 ('지르', 0.6451147794723511),
 ('터뜨릴', 0.6450271606445312),
 ('흐느끼', 0.6383712291717529)]

In [33]:
model_all.wv.similar_by_word("남아")

[('여아', 0.8449685573577881),
 ('초산부', 0.645108699798584),
 ('정상아', 0.6342120170593262),
 ('경산부', 0.6270981431007385),
 ('3540', 0.6189278364181519),
 ('보였으나', 0.616357684135437),
 ('출생체중', 0.6121364235877991),
 ('정상인', 0.6107046604156494),
 ('011', 0.6100091934204102),
 ('남근기', 0.6090734601020813)]

In [34]:
model_all.wv.similar_by_word("여아")

[('남아', 0.8449686169624329),
 ('여자', 0.6180329918861389),
 ('인치', 0.6134419441223145),
 ('617', 0.6080763339996338),
 ('584', 0.6044162511825562),
 ('547', 0.5849400162696838),
 ('남자', 0.583122730255127),
 ('485', 0.5826152563095093),
 ('011', 0.5790334343910217),
 ('여학생', 0.5766470432281494)]

---

## Only Noun

- 이번에는 말뭉치에서 명사만 추출하여 결과를 비교해보자
- 유사한 키워드 잡기를 목적으로 할 때는 명사만 추출하는 것이 더 적절한 듯

In [35]:
%%time
def extract_noun(text):
    tokenizer = Mecab()
    text = tokenizer.nouns(text)
#     text = ' '.join(text)
    return text
    
df['noun'] = df['text'].apply(extract_noun)
df = df.dropna()

CPU times: user 41.9 s, sys: 21.1 s, total: 1min 3s
Wall time: 1min 1s


In [36]:
noun_corpus = np.array(df.noun)

In [37]:
model_noun = Word2Vec(noun_corpus, size=100, workers=4, sg=1)
model_noun.save('data/tokmom_word2vec_only_noun')

In [38]:
"육아" in model_noun.wv.vocab

True

In [39]:
model_noun.wv.similar_by_word("육아")

[('휴직', 0.6748740077018738),
 ('양육', 0.646068811416626),
 ('워킹', 0.6446343064308167),
 ('휴직급', 0.6235928535461426),
 ('핸드북', 0.6208047866821289),
 ('노동법', 0.6126154065132141),
 ('고충', 0.6120660305023193),
 ('유연', 0.6106094121932983),
 ('정주희', 0.603817880153656),
 ('프렌드', 0.6036721467971802)]

In [40]:
model_noun.wv.similar_by_word("시댁")

[('친정', 0.8366267681121826),
 ('시어머니', 0.8245119452476501),
 ('시부모', 0.8139957189559937),
 ('식구', 0.8086997866630554),
 ('친정어머니', 0.803844690322876),
 ('며느리', 0.8013578653335571),
 ('처가', 0.7952950596809387),
 ('남편', 0.794093132019043),
 ('친가', 0.7833516597747803),
 ('장모', 0.7786191701889038)]

In [41]:
model_noun.wv.similar_by_word("놀이")

[('소꿉', 0.7528002858161926),
 ('공놀이', 0.733434796333313),
 ('놀잇감', 0.7070861458778381),
 ('음률', 0.687225341796875),
 ('놀이터', 0.6719777584075928),
 ('유희', 0.6688657999038696),
 ('역할극', 0.6673043370246887),
 ('비석', 0.662082850933075),
 ('미술', 0.6585705280303955),
 ('술래잡기', 0.6578928232192993)]

In [42]:
model_noun.wv.similar_by_word("아이")

[('함정', 0.7817701101303101),
 ('노심', 0.7785893678665161),
 ('게으름', 0.7765206098556519),
 ('취감', 0.775086522102356),
 ('거리감', 0.7740968465805054),
 ('반항기', 0.7687433362007141),
 ('의기소침', 0.766459047794342),
 ('무덤덤', 0.7661758661270142),
 ('응석받이', 0.7661656737327576),
 ('일임', 0.765544056892395)]

In [43]:
model_noun.wv.similar_by_word("치아")

[('잇몸', 0.8551942110061646),
 ('영구치', 0.8134870529174805),
 ('젖니', 0.8032926321029663),
 ('충치', 0.791760265827179),
 ('덧니', 0.7541983127593994),
 ('사랑니', 0.7435067296028137),
 ('앞니', 0.7414108514785767),
 ('어금니', 0.7392208576202393),
 ('유치', 0.7391830682754517),
 ('과잉치', 0.738880455493927)]

In [44]:
model_noun.wv.similar_by_word("이빨")

[('혓바닥', 0.8635708093643188),
 ('속살', 0.8388662338256836),
 ('거립니다', 0.8224554061889648),
 ('혓바늘', 0.8199008107185364),
 ('씹', 0.8177372217178345),
 ('이쑤시개', 0.8059817552566528),
 ('씹기', 0.8043206930160522),
 ('부리', 0.7985096573829651),
 ('입가', 0.7983756065368652),
 ('승환', 0.796153724193573)]

In [45]:
model_noun.wv.similar_by_word("감기")

[('코감기', 0.8609760999679565),
 ('목감기', 0.809931218624115),
 ('편도염', 0.8092349171638489),
 ('기관지염', 0.8082438111305237),
 ('환절기', 0.7973124980926514),
 ('콧물', 0.7859431505203247),
 ('호흡기', 0.7802457809448242),
 ('인후염', 0.7786545753479004),
 ('기침', 0.7666497230529785),
 ('독감', 0.7636998891830444)]

In [46]:
model_noun.wv.similar_by_word("열")

[('고열로', 0.729784369468689),
 ('외감', 0.7267425656318665),
 ('사병', 0.7214865684509277),
 ('熱', 0.7119229435920715),
 ('오한', 0.708365261554718),
 ('허열', 0.7079216837882996),
 ('고열', 0.6981440782546997),
 ('콧물감기', 0.688435971736908),
 ('목감기', 0.6771934628486633),
 ('습과', 0.6768776178359985)]

In [47]:
model_noun.wv.similar_by_word("아파")

[('거립니다', 0.8690185546875),
 ('목도', 0.8589751124382019),
 ('무의식중', 0.8553606271743774),
 ('참기', 0.8429911732673645),
 ('운지', 0.8407555222511292),
 ('청진', 0.8329979181289673),
 ('어린애', 0.831182599067688),
 ('직행', 0.8264451026916504),
 ('속수무책', 0.8260210156440735),
 ('색도', 0.8237227201461792)]

In [48]:
model_noun.wv.similar_by_word("울음")

[('지르기', 0.6825604438781738),
 ('소리', 0.6751922965049744),
 ('비명', 0.6709635257720947),
 ('손짓', 0.6630816459655762),
 ('옹알이', 0.658660352230072),
 ('잠꼬대', 0.6556703448295593),
 ('떼', 0.6556562781333923),
 ('고음', 0.6515142917633057),
 ('신음', 0.6505814790725708),
 ('성어', 0.6473187208175659)]

In [49]:
model_noun.wv.similar_by_word("남아", topn=20)

[('여아', 0.8805588483810425),
 ('음모', 0.7361984252929688),
 ('가슴둘레', 0.7251384258270264),
 ('변성기', 0.721394419670105),
 ('남자', 0.7136530876159668),
 ('남근기', 0.7071888446807861),
 ('액모', 0.7068333625793457),
 ('음경', 0.6954772472381592),
 ('등신', 0.6931884288787842),
 ('아도', 0.6867566108703613),
 ('생시', 0.6816847324371338),
 ('허리둘레', 0.678606390953064),
 ('체격', 0.6770161390304565),
 ('여자', 0.6725210547447205),
 ('구성비', 0.6692276000976562),
 ('증후', 0.6684051752090454),
 ('항문기', 0.6671019792556763),
 ('초경', 0.6670586466789246),
 ('아포크린샘', 0.6651873588562012),
 ('멍울', 0.6637452840805054)]

In [50]:
model_noun.wv.similar_by_word("여아", topn=20)

[('남아', 0.8805588483810425),
 ('음모', 0.6818540096282959),
 ('가슴둘레', 0.6795029640197754),
 ('남근기', 0.662116527557373),
 ('여자', 0.660022497177124),
 ('변성기', 0.655205488204956),
 ('등신', 0.6426079273223877),
 ('항문기', 0.6396911144256592),
 ('액모', 0.63859623670578),
 ('남자', 0.6375669240951538),
 ('생시', 0.6359604597091675),
 ('아도', 0.6350091695785522),
 ('둘레', 0.6200997829437256),
 ('세경', 0.6181658506393433),
 ('조숙', 0.6172573566436768),
 ('음경', 0.6144160032272339),
 ('아일', 0.6128727197647095),
 ('멍울', 0.6122650504112244),
 ('남학생', 0.6102195978164673),
 ('남근', 0.6069267988204956)]

In [51]:
model_noun.wv.similar_by_word("여자", topn=20)

[('남자', 0.8741194009780884),
 ('남아', 0.6725210547447205),
 ('여아', 0.660022497177124),
 ('아이든', 0.6500554084777832),
 ('태생', 0.6371405124664307),
 ('씩', 0.6348239183425903),
 ('남학생', 0.6147850751876831),
 ('군대', 0.6132354736328125),
 ('엘렉트라', 0.6108006834983826),
 ('성별', 0.6100404262542725),
 ('오이디푸스', 0.6061607003211975),
 ('성', 0.6013287305831909),
 ('야동', 0.6008148193359375),
 ('이분법', 0.5963730216026306),
 ('경쟁자', 0.591517984867096),
 ('동경', 0.5906028747558594),
 ('남근', 0.5888799428939819),
 ('여학생', 0.5860785245895386),
 ('근육질', 0.5840975046157837),
 ('동갑내기', 0.5836182236671448)]

In [52]:
model_noun.wv.similar_by_word("남자", topn=20)

[('여자', 0.8741194009780884),
 ('남아', 0.7136530876159668),
 ('씩', 0.7083884477615356),
 ('아이든', 0.6729774475097656),
 ('태생', 0.6685231328010559),
 ('경쟁자', 0.6650919914245605),
 ('군대', 0.6548209190368652),
 ('아버지', 0.6542489528656006),
 ('이분법', 0.6531455516815186),
 ('남근', 0.6515460014343262),
 ('위엄', 0.6440874338150024),
 ('성기', 0.6414923071861267),
 ('성별', 0.6401675939559937),
 ('엘렉트라', 0.6401489973068237),
 ('여아', 0.6375669240951538),
 ('오이디푸스', 0.6355557441711426),
 ('동성', 0.632432758808136),
 ('남학생', 0.6308331489562988),
 ('고정관념', 0.628714382648468),
 ('탈선', 0.6272584199905396)]

In [53]:
model_noun.wv.similar_by_word("유모차")

[('디럭스', 0.8051555156707764),
 ('절충', 0.791560709476471),
 ('휴대용', 0.7657305598258972),
 ('쇼퍼', 0.7534667253494263),
 ('타보', 0.7531963586807251),
 ('페도라', 0.7363781929016113),
 ('핸들링', 0.7304482460021973),
 ('스콥', 0.721701979637146),
 ('포투', 0.7204662561416626),
 ('잉글', 0.7172436118125916)]

In [54]:
model_noun.wv.similar_by_word("카시트")

[('장착', 0.7554982304573059),
 ('안전벨트', 0.7396782040596008),
 ('브라이', 0.7333618402481079),
 ('순성', 0.7287185788154602),
 ('페도라', 0.7266130447387695),
 ('픽스', 0.7214741110801697),
 ('콩코드', 0.7028893232345581),
 ('벨트', 0.6944077610969543),
 ('뒷좌석', 0.6863318681716919),
 ('부스터', 0.6818766593933105)]

### Volume of Corpus 

In [55]:
df[['text']].to_csv('data/text_for_word2vec.csv', index=False, encoding='utf-8')

In [56]:
pd.read_csv('data/text_for_word2vec.csv')

,text
0,6살 남아 입니다 회사 복직 후 3살부터 주중에는 외할머니댁에서 2살 4살 터울의 ...
1,요즘 들어 아빠가 아이에게 다가가려고 하면 엄마한테 갈 거야 하면서 엄마를 유독 찾...
2,둘째가 태어나면 첫째가 힘들 거라는 건 예상하고 있었지만 툭하면 삐치고 울고 좀처럼...
3,20개월 남아가 있고 임신 8개월차로 두 아기의 엄마입니다 첫 아이는 잠자리 독립을...
4,14개월 남자 아기인데 심하게 깨무는 버릇이 있어요 전에는 졸릴 때 주로 물곤 했는...
...,...
19502,토끼띠의 단점
19503,국가영어능력평가 준비 올해부터 본격적으로 시행되는 국가영어능력평가에 대해서 소개해드...
19504,워킹맘이든 전업맘이든 아이들에게 관심과 사랑을 쏟아야 하는 것은 분명합니다 평소 아...
19505,검사결과 지민이의 강점지능은 언어지능과 음악지능였습니다 엄마는 지민이가 집에서나 학...


### Loading check

In [57]:
model = Word2Vec.load('data/tokmom_word2vec_only_noun')
model

In [58]:
model.wv.similar_by_word("집")

[('집안', 0.7387545704841614),
 ('집들이', 0.7016651630401611),
 ('방', 0.6618398427963257),
 ('발품', 0.6376394033432007),
 ('만희', 0.6343194842338562),
 ('현관', 0.6282860636711121),
 ('살림살이', 0.6201419830322266),
 ('가정집', 0.6191257834434509),
 ('녹초', 0.6188140511512756),
 ('난장판', 0.6187115907669067)]

In [59]:
df

,text,tokenized,noun
0,6살 남아 입니다 회사 복직 후 3살부터 주중에는 외할머니댁에서 2살 4살 터울의 ...,"[6, 살, 남, 아, 입, 니, 다, 회사, 복직, 후, 3, 살, 부터, 주, ...","[살, 회사, 복직, 후, 살, 주, 중, 외할머니, 댁, 살, 살, 터울, 사촌,..."
1,요즘 들어 아빠가 아이에게 다가가려고 하면 엄마한테 갈 거야 하면서 엄마를 유독 찾...,"[요즘, 들, 어, 아빠, 가, 아이, 에게, 다가가, 려고, 하, 면, 엄마, 한...","[아빠, 아이, 엄마, 거, 엄마, 전, 정도, 엄마, 아빠, 거부, 엄마, 눈, ..."
2,둘째가 태어나면 첫째가 힘들 거라는 건 예상하고 있었지만 툭하면 삐치고 울고 좀처럼...,"[둘째, 가, 태어나, 면, 첫째, 가, 힘들, 거, 라는, 건, 예상, 하, 고,...","[둘째, 첫째, 거, 건, 예상, 때, 엄마, 껌, 딱지, 첫째, 외출, 때, 제외..."
3,20개월 남아가 있고 임신 8개월차로 두 아기의 엄마입니다 첫 아이는 잠자리 독립을...,"[20, 개월, 남아, 가, 있, 고, 임신, 8, 개월, 차, 로, 두, 아기, ...","[개월, 남아, 임신, 개월, 차, 아기, 엄마, 아이, 잠자리, 독립, 생후, 일..."
4,14개월 남자 아기인데 심하게 깨무는 버릇이 있어요 전에는 졸릴 때 주로 물곤 했는...,"[14, 개월, 남자, 아기, 인데, 심하, 게, 깨무, 는, 버릇, 이, 있, 어...","[개월, 남자, 아기, 버릇, 전, 때, 이제, 자기, 지대, 때, 떼, 상대방, ..."
...,...,...,...
19502,토끼띠의 단점,"[토끼띠, 의, 단점]","[토끼띠, 단점]"
19503,국가영어능력평가 준비 올해부터 본격적으로 시행되는 국가영어능력평가에 대해서 소개해드...,"[국가, 영어, 능력, 평가, 준비, 올해, 부터, 본격, 적, 으로, 시행, 되,...","[국가, 영어, 능력, 평가, 준비, 올해, 본격, 시행, 국가, 영어, 능력, 평..."
19504,워킹맘이든 전업맘이든 아이들에게 관심과 사랑을 쏟아야 하는 것은 분명합니다 평소 아...,"[워킹, 맘, 이, 든, 전업, 맘, 이, 든, 아이, 들, 에게, 관심, 과, 사...","[워킹, 맘, 전업, 맘, 아이, 관심, 사랑, 것, 평소, 아이, 생각, 것, 관..."
19505,검사결과 지민이의 강점지능은 언어지능과 음악지능였습니다 엄마는 지민이가 집에서나 학...,"[검사, 결과, 지민, 이, 의, 강점, 지능, 은, 언어, 지능, 과, 음악, 지...","[검사, 결과, 지민, 강점, 지능, 언어, 지능, 음악, 지능, 엄마, 지민, 집..."


## Doc2Vec

In [60]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [61]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(df['noun'])]

In [62]:
%%time
doc2vec_model = Doc2Vec(documents, 
                        vector_size=100, 
                        window=5, 
                        min_count=5, 
                        workers=4)

model.save('data/tokmom_doc2vec_only_noun.d2v')

CPU times: user 42.6 s, sys: 1.09 s, total: 43.7 s
Wall time: 19.5 s


In [63]:
len(doc2vec_model.docvecs)

19507

In [64]:
df['text'][0]

'6살 남아 입니다 회사 복직 후 3살부터 주중에는 외할머니댁에서 2살 4살 터울의 사촌 형 누나와 함께 자랐어요 금요일 저녁에 데려와 주말에는 엄마 아빠와 생활하고 있구요 7세가 되는 내년이라도 아이를 데려오자는 아이 아빠의 의견과 유치원과 태권도 등 패턴이 적응되었고 형 누나들과 더 자랄 수 있게 초등학교 입학시기에 데리고 가는게 낫지 않겠냐는 외할머니 의견이 다릅니다 6세가 된 아이의 교육과 생활습관 형성이 이제는 할머니댁에서는 어려울 것 같다는 생각이 들고 하루하루 생각이 크고 자라는 아이를 주말에만 보는 저 또한 놀랄 때가 많거든요 하루라도 빨리 데려와야 할까요 초등학교 입학시기에 데려오는 편이 아이에게 좋을까요 매우 어려운 질문입니다 먼저 아이가 현재 잘 적응하고 있고 주변 양육환경 역시 아이가 성장하기에 별다른 문제가 없다 하니 고민이 더욱 크시리라 생각됩니다 우선 되어야하는 것은 아이의 의견입니다 6살이면 이러한 문제에 대하여 스스로 생각하고 의견을 전달할 수 있으므로 먼저 아이의 의견을 들어 주시기 바랍니다 아이는 부모님과 함께 사는 것이 좋습니다 그러나 부모님께서 두분 모두 직장에 나가시니 유치원 하원 후 아이를 돌보아 주실 분이 현재 함께 생활하고 계시는 할머님이시면 재고의 여지 없이 데려 오시는 것을 권유 드립니다 그러나 하원 후 할머님의 돌봄이 여의치 못할 경우 돌보아 줄 대상이 당분간 할머님댁에서 시간제로나마 아이를 돌보아 주어 아이와의 상호작용을 충분히 가진 후에 아이가 집으로 왔을 때 유치원 하원 후에 계속 돌보아 줄 수 있으면 좋습니다 이와 같이 먼저 아이가 집으로 온 후 부모님께서 퇴근하시기 전 까지 돌보아 줄 사람을 알아보시고 데려 오시는 것이 좋다는 생각이 듭니다'

In [65]:
print("Input Document", df['text'][100], sep="\n")

similar_docs = doc2vec_model.docvecs.most_similar(100)

print("Most similar Documents")
for idx, similarity in similar_docs[:3]:
    print("idx:", idx, "similarity:", np.round(similarity, 3))
    print(df['text'][idx])

Input Document
젖병을 떼야 하는 이유로는 첫째 우식증을 예방하고 둘째 젖병 무는 어린 아기의 습관을 서서히 고쳐 나가기 위함 입니다 지금은 단지 습관에 의해 젖병을 물고 있을 뿐이므로 음식은 숟가락으로 먹이고 우유는 컵으로 마시게 하면서 젖병을 뗄 수 있도록 해야 합니다 잠잘 때 젖병을 물고 자는 것은 무의식적인 행동입니다 그러나 너무 친숙해진 습관을 갑자기 금지시킨다면 아기 심리에 큰 충격을 줄 수도 있으므로 서서히 젖병 속의 우유를 희석시키는 방법을 사용합니다 예를 들어 평상시에 우유를 200 mL 먹였다면 우유 180 mL에 물 20 mL를 섞어서 젖병에 넣어줍니다 점차 물의 농도를 높여가다가 아기가 이를 느끼고 부정적인 반응을 보일 때는 잠시 정지하였다가 다시 희석의 강도를 높여봅니다 그러다 보면 아기는 점차 희석된 우유에 흥미를 잃어 스스로 젖병을 내던지게 될 것입니다 아기에 따라 스스로 정서적 안정을 얻는 습관을 하나씩 가지고 있습니다 아기 때의 이불을 늘 잡고 있다거나 엄마의 머리카락을 잡아야 잠을 잔다거나 하는 습관은 고치기가 쉽지 않은 일입니다 젖병도 그런 습관의 일종일 수 있습니다 젖병을 물고 있는 것이 정서적인 이유라면 젖병 떼기를 강요하지 않습니다 하루 2 3번은 젖병을 물고 있게 하다가 점차 아기가 성장하면서 젖병 무는 것이 자신에게는 어울리지 않는다는 것을 엄마가 끊임없이 설득하고 아이 스스로도 그렇게 느껴 자연스레 젖병을 멀리하게 합니다 가능한 컵이나 빨대컵을 활용하고 노리개 젖꼭지 등을 대체하여 사용해 볼 수도 있습니다
Most similar Documents
idx: 14146 similarity: 0.669
모든 아이들은 만 2세까지 빠는 습관을 가지고 있다 아이들이 무엇인가를 빠는 이유는 무엇일까 스스로를 달래기 위함이다 젖을 물리면 아이들은 자신의 손가락이나 다른 물건을 빨면서 불안한 마음을 달래게 되는 것이다 이럴 경우 사용하는 것이 바로 공갈젖꼭지 일명 뽁뽁이이다 하지만 초보 엄마는 언제 공갈젖꼭지를 물려야 

In [66]:
df['tokenized_doc'] = df['noun'].apply(lambda x: doc2vec_model.infer_vector(x).astype(float))

In [67]:
df

,text,tokenized,noun,tokenized_doc
0,6살 남아 입니다 회사 복직 후 3살부터 주중에는 외할머니댁에서 2살 4살 터울의 ...,"[6, 살, 남, 아, 입, 니, 다, 회사, 복직, 후, 3, 살, 부터, 주, ...","[살, 회사, 복직, 후, 살, 주, 중, 외할머니, 댁, 살, 살, 터울, 사촌,...","[0.08910741657018661, 0.09750863164663315, 0.4..."
1,요즘 들어 아빠가 아이에게 다가가려고 하면 엄마한테 갈 거야 하면서 엄마를 유독 찾...,"[요즘, 들, 어, 아빠, 가, 아이, 에게, 다가가, 려고, 하, 면, 엄마, 한...","[아빠, 아이, 엄마, 거, 엄마, 전, 정도, 엄마, 아빠, 거부, 엄마, 눈, ...","[0.2992932200431824, 0.15961413085460663, 0.42..."
2,둘째가 태어나면 첫째가 힘들 거라는 건 예상하고 있었지만 툭하면 삐치고 울고 좀처럼...,"[둘째, 가, 태어나, 면, 첫째, 가, 힘들, 거, 라는, 건, 예상, 하, 고,...","[둘째, 첫째, 거, 건, 예상, 때, 엄마, 껌, 딱지, 첫째, 외출, 때, 제외...","[0.08892737329006195, 0.1796158254146576, 0.29..."
3,20개월 남아가 있고 임신 8개월차로 두 아기의 엄마입니다 첫 아이는 잠자리 독립을...,"[20, 개월, 남아, 가, 있, 고, 임신, 8, 개월, 차, 로, 두, 아기, ...","[개월, 남아, 임신, 개월, 차, 아기, 엄마, 아이, 잠자리, 독립, 생후, 일...","[0.03297119215130806, 0.16492007672786713, 0.4..."
4,14개월 남자 아기인데 심하게 깨무는 버릇이 있어요 전에는 졸릴 때 주로 물곤 했는...,"[14, 개월, 남자, 아기, 인데, 심하, 게, 깨무, 는, 버릇, 이, 있, 어...","[개월, 남자, 아기, 버릇, 전, 때, 이제, 자기, 지대, 때, 떼, 상대방, ...","[0.08187475055456161, 0.08348076790571213, 0.0..."
...,...,...,...,...
19502,토끼띠의 단점,"[토끼띠, 의, 단점]","[토끼띠, 단점]","[0.006086952984333038, 0.035688772797584534, 0..."
19503,국가영어능력평가 준비 올해부터 본격적으로 시행되는 국가영어능력평가에 대해서 소개해드...,"[국가, 영어, 능력, 평가, 준비, 올해, 부터, 본격, 적, 으로, 시행, 되,...","[국가, 영어, 능력, 평가, 준비, 올해, 본격, 시행, 국가, 영어, 능력, 평...","[-0.023202231153845787, 0.09385380148887634, 0..."
19504,워킹맘이든 전업맘이든 아이들에게 관심과 사랑을 쏟아야 하는 것은 분명합니다 평소 아...,"[워킹, 맘, 이, 든, 전업, 맘, 이, 든, 아이, 들, 에게, 관심, 과, 사...","[워킹, 맘, 전업, 맘, 아이, 관심, 사랑, 것, 평소, 아이, 생각, 것, 관...","[0.011682785116136074, 0.10789419710636139, 0...."
19505,검사결과 지민이의 강점지능은 언어지능과 음악지능였습니다 엄마는 지민이가 집에서나 학...,"[검사, 결과, 지민, 이, 의, 강점, 지능, 은, 언어, 지능, 과, 음악, 지...","[검사, 결과, 지민, 강점, 지능, 언어, 지능, 음악, 지능, 엄마, 지민, 집...","[-0.2181546986103058, 0.12740351259708405, 0.1..."
